In [1]:
import boto3
import os

In [ ]:
access_key_id = "minioadmin"
secret_access_key = "minioadmin"
minio_url = "http://localhost:9000"

minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

new_bucket = "formatted-zone"
try:
    minio_client.head_bucket(Bucket=new_bucket)
except:
    minio_client.create_bucket(Bucket=new_bucket)
    
    
    
name_bucket = "dump"
try:
    minio_client.head_bucket(Bucket=name_bucket)
    print(f"Bucket '{bucket}' ja existeix")
except:
    minio_client.create_bucket(name_bucket)

In [ ]:
from pydub import AudioSegment
import io
from tqdm import tqdm

bucket_origen = "persistent-landing"
bucket_desti = "formatted-zone"
prefix_origen = "audios/"

# Formatos de audio soportados
formatos_audio = {'.mp3', '.wav', '.flac', '.m4a', '.aac', '.ogg', '.wma'}

paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_origen, Prefix=prefix_origen):
    for obj in tqdm(page.get("Contents", []), desc="Processant àudios"):
        key = obj["Key"]
        filename = key.split("/")[-1]
        
        # Obtener la extensión del archivo
        extension = os.path.splitext(filename)[1].lower()
        
        # Solo procesar archivos de audio
        if extension not in formatos_audio:
            # Guardar archivo no reconocido en dump
            response = minio_client.get_object(Bucket=bucket_origen, Key=key)
            file_data = response["Body"].read()
            
            minio_client.put_object(
                Bucket="dump",
                Key=key,
                Body=file_data
            )
            continue
            
        # Crear el nuevo nombre con extensión .mp3
        nombre_sin_extension = os.path.splitext(filename)[0]
        nuevo_filename = f"{nombre_sin_extension}.mp3"

        # Llegir àudio original
        response = minio_client.get_object(Bucket=bucket_origen, Key=key)
        audio_data = response["Body"].read()

        # Obrir amb pydub (detectar formato automáticamente)
        try:
            audio = AudioSegment.from_file(io.BytesIO(audio_data))
        except Exception as e:
            print(f"Error amb {filename}: {e}")
            continue

        # Desa com MP3 i puja al bucket formatted-zone
        buffer = io.BytesIO()
        audio.export(buffer, format="mp3")
        buffer.seek(0)

        new_key = f"audios/{nuevo_filename}"
        minio_client.put_object(
            Bucket=bucket_desti,
            Key=new_key,
            Body=buffer
        )

Processant àudios:   0%|          | 0/17 [00:00<?, ?it/s]

Processant àudios: 100%|██████████| 17/17 [00:15<00:00,  1.07it/s]
